In [13]:
import numpy as np
import pandas as pd
import csv
from datetime import datetime, timedelta
from dataset_utils import read_numpy_from_file, numpy_to_pandas, output_pandas_to_file
from baseline_datasets.merge_baseline_dataset import get_72h_signal_issuance, distance_to_HK, check_direct_strike

### Helper functions

In [2]:
# cannot import because of the rearrangement of the folder, but ok
from geographiclib.geodesic import Geodesic

def azimuth_from_HK(lat: float, long: float):
    '''Takes in a latitude and a longitude, returns its forward azimuth from Hong Kong.'''
    # constants
    hko_lat = 22.302219
    hko_long = 114.174637
    azimuth = Geodesic.WGS84.Inverse(hko_lat, hko_long, lat, long)['azi1']
    # the given azimuth is in [-180, 180], I prefer [0, 360] instead
    return (azimuth + 360.0) % 360.0

In [3]:
def get_immediate_signal_issuance(name: str, year: int, month: int, day: int, hour: int, issuance: pd.DataFrame, signal: int):
    '''
    Checks if the specified signal is being hoisted at the same time given these information:

    Parameters:
        name (str): TC name
        year (int): season
        month (int): month value
        day (int): day value
        hour (int): hour value
        issuance (pandas dataframe): HKO issuance records
        signal (int): the target signal, either 1, 3, or 8.

    Returns:
        result (bool): True if hoisted right now, False otherwise
    '''

    # handle name anomalies: merged storms and poorly tracked storms due to primitive technology
    if name == 'FLOSSIE:GRACE': return False # 1950/1966/1969, never affected HK despite the 3 incarnations
    elif name == 'LUCRETIA:NANCY': return False # 1950, same as above
    elif name == 'JEANNE:JEANNIE': return False # 1952, same as above, different names for the same TC
    elif name == 'KAREN:LUCILLE': return False # 1956, these two merged approx. 900km away from HK and never affected HK
    elif name == 'ANITA:WILDA': name = 'WILDA' # 1959, WILDA gave HK a T1
    elif name == 'OPAL:RUTH': return False # 1959, never affected HK
    elif name == 'NORA:PATSY': name = 'NORA' # 1959, NORA gave HK a nice T3
    elif name == 'BABE:BABS:CARLA' or name == 'BABE:CARLA:CHARLOTTE:CARLA': return False # 1962, never affected HK, naming shenanigans intensify
    elif name == 'EMMA:FREDA' or name == 'FREDA:GILDA': return False # 1962, never affected HK. They must have had a hard time tracking close TCs in 1962
    elif name == 'GILDA:IVY': return False # 1962, separate line in memory of Ivy who was murdered by the cannibal Gilda
    elif name == 'FRAN:GEORGIA' and year == 1964: name = 'GEORGIA' # 1964, GEORGIA gave HK a T1
    elif name == 'LOUISE:MARGE': return False # 1964, never affected HK, merged
    elif name == 'IVY:JEAN':  return False # 1965, never affected HK
    elif name == 'FRAN:GEORGIA' and year == 1967: name = 'FRAN' # 1967, this cursed pair again. FRAN led to TWO separate T1's.
    elif name == 'BILIE:BILLIE': return False # 1970, this never affected HK, different names for the same thing
    elif name == 'FAYE(GLORIA):GLORIA': return False # 1971, never affected HK, Faye annexed by Gloria early on
    elif name == 'HELEN:HELLEN': return False # 1975, never affected HK, different names
    elif name == 'BESS:BONNIE': name == 'BONNIE' # 1978, T3 in August
    elif name == 'TESS:VAL': name = 'TESS' # 1982, T3, VAL succeeded TESS
    elif name == 'KEN-LOLA:LOLA': return False # 1989, poorly organized TC, hit Shanghai but not HK
    elif name == 'PAT:RUTH': return False # 1994, did not affect HK, merged
    elif name == 'ABEL:BETH': name = 'BETH' # 1996, Beth gave HK a T1
    elif name == 'ORAJI:TORAJI': return False # 2018, but Oraji is a typo of Toraji
    elif name == 'BULBUL:MATMO': return False # 2019, Matmo went to the Indian Ocean and became Bulbul. No impact on HK

    if name == 'NOT_NAMED': name = 'no name' # naming conventions difference

    # select relevant HKO records and then narrow down the search
    if signal == 1:
        signals = [1]
    elif signal == 3:
        signals = [3]
    elif signal == 8:
        signals = [8, 9, 10]
    else: return None
    records = issuance.query("(Name == @name) and (StartYY == @year) and (Signal in @signals)")
    if records.shape[0] < 1: return False # this TC never led to any signals

    time_to_check = datetime(year, month, day, hour, 0)
    # for each potential match, check if the given current time is within the start-end period
    for _, row in records.iterrows():
        startTime = datetime(year, row['StartMM'], row['StartDD'], row['StartHH'], row['Startmm'])
        endTime = datetime(year, row['EndMM'], row['EndDD'], row['EndHH'], row['Endmm'])

        if startTime <= time_to_check <= endTime:
            return True

    return False

In [4]:
def read_csv_to_dict(filename):
    '''Takes in a filename to a CSV file (without extension) and returns its content as a dictionary (str -> float)'''
    with open(filename, 'r') as infile:
        reader = csv.reader(infile)
        mydict = {rows[0]:float(rows[1]) for rows in reader}
    return mydict

### Read data sources

In [5]:
# Read dynamical data
easm = read_csv_to_dict("./dynamical_data/easm_indices.csv")
hi_humid = read_csv_to_dict("./dynamical_data/hi_humid.csv")
hk_u_winds = read_csv_to_dict("./dynamical_data/hk_u_winds.csv")
lo_humid = read_csv_to_dict("./dynamical_data/lo_humid.csv")
hk_v_winds = read_csv_to_dict("./dynamical_data/hk_v_winds.csv")
mlvws = read_csv_to_dict("./dynamical_data/mlvms.csv") # fix typo!
pott = read_csv_to_dict("./dynamical_data/pott.csv")
temp_surface = read_csv_to_dict("./dynamical_data/temp_surface.csv")
temp200 = read_csv_to_dict("./dynamical_data/temp200.csv")
u200 = read_csv_to_dict("./dynamical_data/u200.csv")
u500 = read_csv_to_dict("./dynamical_data/u500.csv")
ulvws = read_csv_to_dict("./dynamical_data/ulvms.csv") # fix typo!
v500 = read_csv_to_dict("./dynamical_data/v500.csv")
vort850 = read_csv_to_dict("./dynamical_data/vort850.csv")
westerly = read_csv_to_dict("./dynamical_data/westerly_indices.csv")
wnpsh_area = read_csv_to_dict("./dynamical_data/wnpsh_area_indices.csv")
wnpsh_intensity = read_csv_to_dict("./dynamical_data/wnpsh_intensity_indices.csv")
wnpsh_extension = read_csv_to_dict("./dynamical_data/wnpsh_extension_indices.csv")

In [6]:
# Read the other data
issuance = numpy_to_pandas(read_numpy_from_file('./traditional_data/issuance.npy'))
best_track = numpy_to_pandas(read_numpy_from_file('./traditional_data/best_track.npy'))
best_track = best_track.query("(SEASON > 1999) or (SEASON == 1999 and ISO_TIME_MONTH > 7)")
best_track.describe()

Reading file ./traditional/issuance.npy ......Done!
Reading file ./traditional/best_track.npy ......Done!


,SEASON,ISO_TIME_MONTH,ISO_TIME_DAY,ISO_TIME_HOUR,ISO_TIME_MIN,USA_LAT,USA_LON,USA_WIND,USA_PRES,USA_R34_NE,...,USA_R50_NE,USA_R50_SE,USA_R50_SW,USA_R50_NW,USA_R64_NE,USA_R64_SE,USA_R64_SW,USA_R64_NW,STORM_SPEED,STORM_DIR
count,15033.000000,15033.000000,15033.000000,15033.000000,15033.0,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,...,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000,15033.000000
mean,2009.038914,8.160247,15.702122,8.980975,0.0,18.981850,131.691559,54.421938,-7429.102441,-43797.865562,...,-67676.852325,-67698.088539,-67872.633806,-67797.964146,-77832.682898,-77899.752944,-77953.541209,-77912.947316,10.168496,234.193441
std,6.187875,2.341304,8.666252,6.716750,0.0,7.349080,19.086874,32.503149,27902.697658,49680.806222,...,46793.468148,46784.069208,46715.952033,46746.058339,41548.057650,41502.381977,41465.478383,41493.640483,5.443846,113.888762
min,1999.000000,1.000000,1.000000,0.000000,0.0,1.300000,-180.000000,10.000000,-99999.000000,-99999.000000,...,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,0.000000,0.000000
25%,2003.000000,7.000000,8.000000,2.000000,0.0,13.600000,121.300003,30.000000,956.000000,-99999.000000,...,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,6.000000,191.000000
50%,2009.000000,8.000000,16.000000,6.000000,0.0,18.400000,130.899994,45.000000,987.000000,60.000000,...,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,9.000000,284.000000
75%,2015.000000,10.000000,23.000000,12.000000,0.0,23.600000,142.000000,75.000000,1000.000000,120.000000,...,36.000000,35.000000,30.000000,35.000000,-99999.000000,-99999.000000,-99999.000000,-99999.000000,13.000000,310.000000
max,2019.000000,12.000000,31.000000,23.000000,0.0,45.099998,179.800003,170.000000,1012.000000,330.000000,...,200.000000,215.000000,205.000000,196.000000,135.000000,120.000000,110.000000,107.000000,52.000000,360.000000


### Building the Dataset

Columns and data types of the combined set:

The first four columns constitute the correct labels.
```
0 : bool MINIMAL_IMPACT, T1 ground truth
1 : bool LIMITED_IMPACT, T3 ground truth
2 : bool SUBSTANTIAL_IMPACT, T8-T10 ground truth
3 : bool DIRECT_STRIKE, direct strike ground truth
```

Then there will be n = (past_track_limit // 6) + 1 repetitions of the following as a time series:

```
0 : int MMxx, month and data of this record in the series
1 : int DDxx
2 : bool MI_STATUSxx, whether T1 is hoisted at this point (MM/DD HH)
3 : bool LI_STATUSxx, whether T3 is hoisted at this point
4 : bool SI_STATUSxx, whether T8-T10 is hoisted at this point
5 : bool DS_STATUSxx, whether there is a direct strike at this point 
6 : float DISTxx, the radial distance of the storm from HK at this point
7 : float AZMxx, azimuth of the storm from HK
8 : int SPEEDxx, storm speed
9 : float DIRxx, storm heading bearing
10 : int VMAXxx, storm intensity
11 : int DVMAXxx, change in storm intensity, unavailable for the oldest record in sequence.
12 : float ULVWSxx, Upper-Lower level Vertical Wind Shear, units in m/s
13 : float MLVWSxx, Mid-Lower level Vertical Wind Shear, units in m/s
14 : float HI_HUMIDxx, upper level relative humidity, units in %
15 : float LO_HUMIDxx, mid/lower level relative humidity, units in %
16 : float STEMPxx, Surface TEMPerature, units in K
17 : float UTEMPxx, Upper level TEMPerature, units in K
18 : float U_HKxx, Hong Kong u-wind, units in m/s
19 : float V_HKxx, Hong Kong v-wind, units in m/s
20 : float U200_xx, 200hPa u-wind, m/s
21 : float U500_xx, 500hPa u-wind, m/s
22 : float V500_xx, 500hPa v-wind, m/s
23 : float EASMxx, East Asia Summer Monsoon index, m/s
24 : float VORTxx, 850hPa vorticity, s^-1, scaled up by 1e6
25 : float WESTERLYxx, Westerly index, unit gpm
26 : int SH_AREAxx, western north pacific Subtropical High Area index, no units
27 : float SH_INTxx, WNPSH intensity index, unit gpm
28 : int SH_EXTxx, WNPSP western EXTension index, unit degrees longitude
29 : float POTTxx, POTential Temperature sigma level 0.995, unit K 
```

The symbol xx stands for the number of hours behind the start of the sequence,
i.e. xx in \[0, 6, ..., past_track_limit\].

There should be a total of (30 * (24//6 + 1)) - 1 + 4 = 153 columns in the dataset.

In [7]:
PAST_TRACK_LIMIT = 24

# define column names
columns = [
    # rename columns so they are short enough to work with
    'LOW_IMPACT', 'MID_IMPACT', 'BIG_IMPACT', 'DIRECT_STRIKE'
]
for i in range(0, PAST_TRACK_LIMIT+6, 6):
    columns.append('MM{0:02d}'.format(i))
    columns.append('DD{0:02d}'.format(i))

    columns.append('MI_STATUS{0:02d}'.format(i))
    columns.append('LI_STATUS{0:02d}'.format(i))
    columns.append('SI_STATUS{0:02d}'.format(i))
    columns.append('DS_STATUS{0:02d}'.format(i))

    columns.append('DIST{0:02d}'.format(i))
    columns.append('AZM{0:02d}'.format(i))
    columns.append('SPEED{0:02d}'.format(i))
    columns.append('DIR{0:02d}'.format(i))
    columns.append('VMAX{0:02d}'.format(i))

    if i != PAST_TRACK_LIMIT:
        columns.append('DVMAX{0:02d}'.format(i))
        
    columns.append('ULVWS{0:02d}'.format(i))
    columns.append('MLVWS{0:02d}'.format(i))
    
    columns.append('HI_HUMID{0:02d}'.format(i))
    columns.append('LO_HUMID{0:02d}'.format(i))
    
    columns.append('STEMP{0:02d}'.format(i))
    columns.append('UTEMP{0:02d}'.format(i))
    
    columns.append('U_HK{0:02d}'.format(i))
    columns.append('V_HK{0:02d}'.format(i))    
    columns.append('U200_{0:02d}'.format(i))
    columns.append('U500_{0:02d}'.format(i))
    columns.append('V500_{0:02d}'.format(i))    
    columns.append('EASM{0:02d}'.format(i))
    
    columns.append('VORT{0:02d}'.format(i))
    
    columns.append('WESTERLY{0:02d}'.format(i))
    columns.append('SH_AREA{0:02d}'.format(i))
    columns.append('SH_INT{0:02d}'.format(i))
    columns.append('SH_EXT{0:02d}'.format(i))
    
    columns.append('POTT{0:02d}'.format(i))
    
print(len(columns))

153


In [25]:
storms = best_track['SID'].unique()
print('Number of storms identified in the dataset: {0}'.format(storms.shape[0]))
dataArray = []
print("Creating time series from dynamical data, best track data sequences and warning records.")
num_storms_processed = 0
num_series_ng = 0

# for each unique storm, ....
for storm in storms:
    same_storm = best_track.query('SID == @storm', inplace=False)
    same_storm = same_storm.reset_index(drop=True)

    # generate data rows for the storm
    for index, row in same_storm.iloc[PAST_TRACK_LIMIT//6:].iterrows(): # skip to the first 24 hour time step

        if row['SEASON'] < 1946: continue # no TC warning signals records before 1946 in our data source

        data_row = []
        
        ### true labels ###
        # get signal issuance: already considered next 72 hours
        # but TC w/o names need further consideration
        signal1 = get_72h_signal_issuance(
            row['NAME'], row['SEASON'], row['ISO_TIME_MONTH'], 
            row['ISO_TIME_DAY'], row['ISO_TIME_HOUR'], issuance, 1)
        signal3 = get_72h_signal_issuance(
            row['NAME'], row['SEASON'], row['ISO_TIME_MONTH'], 
            row['ISO_TIME_DAY'], row['ISO_TIME_HOUR'], issuance, 3)
        signal8 = get_72h_signal_issuance(
            row['NAME'], row['SEASON'], row['ISO_TIME_MONTH'], 
            row['ISO_TIME_DAY'], row['ISO_TIME_HOUR'], issuance, 8)
        # direct strike (right now)
        direct_strike = check_direct_strike(row['USA_LAT'], row['USA_LON'])
        
        # select next 72 hours to check:
        #   - if a nameless TC enters 800km radius of HK (likely the signal issuance values above are correct)
        #   - if a TC that is not in 800km radius of HK right now will enter that radius
        #   - if the direct strike situation will change (i.e. TC gets within 100km)
        # if TC dissipates (no more records) in the next 72 hours, just consider as far ahead as possible.
        if row['NAME'] == 'NOT_NAMED' or not direct_strike:
            if (index + 12) > same_storm.shape[0]:
                next_data = same_storm.iloc[index+1:] # choose the rest
            else:
                next_data = same_storm.iloc[index+1: index+13]

            enter_HK_800_km = False
            for idx, item in next_data.iterrows():
                # check if distance ever decreased
                dist = distance_to_HK(item['USA_LAT'], item['USA_LON'])
                if dist <= 800:
                    enter_HK_800_km = True
                    # check if direct strike ground truth changed
                    if dist <= 100:
                        direct_strike = True

            if row['NAME'] == 'NOT_NAMED' and not enter_HK_800_km:
                signal1 = False; signal3 = False; signal8 = False

        data_row.append(signal1)
        data_row.append(signal3)
        data_row.append(signal8)
        data_row.append(direct_strike)
        
        ### Time series from t-0 to t-24, i.e. the past 24/6 = 4 records with the current one
        cannot_build_past_track = False
        for hh in range(0, (PAST_TRACK_LIMIT//6)+1):
            if same_storm.iloc[index-hh]['USA_WIND'] < 0:
                cannot_build_past_track = True
                num_series_ng += 1
                break # skip this whole record (see below), this wind speed value is not available

            ## time information
            data_row.append(same_storm.iloc[index-hh]['ISO_TIME_MONTH'])
            data_row.append(same_storm.iloc[index-hh]['ISO_TIME_DAY'])
            
            ## get current impact status
            signal1 = get_immediate_signal_issuance(
                same_storm.iloc[index-hh]['NAME'], same_storm.iloc[index-hh]['SEASON'], same_storm.iloc[index-hh]['ISO_TIME_MONTH'], 
                same_storm.iloc[index-hh]['ISO_TIME_DAY'], same_storm.iloc[index-hh]['ISO_TIME_HOUR'], issuance, 1)
            data_row.append(signal1)
            signal3 = get_immediate_signal_issuance(
                same_storm.iloc[index-hh]['NAME'], same_storm.iloc[index-hh]['SEASON'], same_storm.iloc[index-hh]['ISO_TIME_MONTH'], 
                same_storm.iloc[index-hh]['ISO_TIME_DAY'], same_storm.iloc[index-hh]['ISO_TIME_HOUR'], issuance, 3)
            data_row.append(signal3)
            signal8 = get_immediate_signal_issuance(
                same_storm.iloc[index-hh]['NAME'], same_storm.iloc[index-hh]['SEASON'], same_storm.iloc[index-hh]['ISO_TIME_MONTH'], 
                same_storm.iloc[index-hh]['ISO_TIME_DAY'], same_storm.iloc[index-hh]['ISO_TIME_HOUR'], issuance, 8)
            data_row.append(signal8)
            direct_strike = check_direct_strike(same_storm.iloc[index-hh]['USA_LAT'], same_storm.iloc[index-hh]['USA_LON'])
            data_row.append(direct_strike)
            
            ## TC best track predictors
            # DIST
            data_row.append(distance_to_HK(same_storm.iloc[index-hh]['USA_LAT'], same_storm.iloc[index-hh]['USA_LON']))
            # AZI
            azimuth = azimuth_from_HK(same_storm.iloc[index-hh]['USA_LAT'], same_storm.iloc[index-hh]['USA_LON'])
            data_row.append(azimuth)
            # SPEED
            data_row.append(same_storm.iloc[index-hh]['STORM_SPEED'])
            # DIR
            data_row.append(same_storm.iloc[index-hh]['STORM_DIR'])
            # VMAX
            vmax = same_storm.iloc[index-hh]['USA_WIND']
            data_row.append(vmax)
            # DVMAX
            if hh != (PAST_TRACK_LIMIT//6):
                old_vmax = same_storm.iloc[index-hh-1]['USA_WIND']
                data_row.append(vmax - old_vmax)
                
            ## Dynamical predictors
            # keys to obtain values from dicts
            time_key = "{0}{1:02d}{2:02d}_{3:02d}_00".format(row['SEASON'], row['ISO_TIME_MONTH'], row['ISO_TIME_DAY'], row['ISO_TIME_HOUR'])
            key = row['SID'] + '_' + time_key
            
            # wind shear
            if key in ulvws:
                data_row.append(ulvws[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break 
            if key in mlvws:
                data_row.append(mlvws[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
                
            # humidity
            if key in hi_humid:
                data_row.append(hi_humid[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
            if key in lo_humid:
                data_row.append(lo_humid[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
                
            # temperature
            if key in temp_surface:
                data_row.append(temp_surface[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
            if key in temp200:
                data_row.append(temp200[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
                
            # u- and v- wind components
            # hk u- and v-winds
            if time_key in hk_u_winds:
                data_row.append(hk_u_winds[time_key])
            else:
                # many best track records are taken at hours not divisible by 6 (e.g. at 1500)
                # interpolation is needed to maximize the utilization of our weather data
                try:
                    # get timestamp
                    timestamp = datetime(row["SEASON"], row["ISO_TIME_MONTH"], row["ISO_TIME_DAY"], row["ISO_TIME_HOUR"], 0, 0)
                    # move back to nearest available record
                    number_of_hours_to_go_back = int(row["ISO_TIME_HOUR"] % 6)
                    timestamp -= timedelta(hours=number_of_hours_to_go_back)
                    value1 = hk_u_winds[timestamp.strftime("%Y%m%d_%H_%M")]
                    # and next available
                    timestamp += timedelta(hours=6)
                    value2 = hk_u_winds[timestamp.strftime("%Y%m%d_%H_%M")]
                    # take weighted sum
                    weight = 1 - (row["ISO_TIME_HOUR"] - (row["ISO_TIME_HOUR"] // 6)*6)/6.0
                    value = value1 * weight + value2 * (1-weight)
                    data_row.append(value)
                except KeyError as e:
                    cannot_build_past_track = True
                    num_series_ng += 1
                    break
            if time_key in hk_v_winds:
                data_row.append(hk_v_winds[time_key])
            else:
                try:
                    timestamp = datetime(row["SEASON"], row["ISO_TIME_MONTH"], row["ISO_TIME_DAY"], row["ISO_TIME_HOUR"], 0, 0)
                    number_of_hours_to_go_back = int(row["ISO_TIME_HOUR"] % 6)
                    timestamp -= timedelta(hours=number_of_hours_to_go_back)
                    value1 = hk_v_winds[timestamp.strftime("%Y%m%d_%H_%M")]
                    timestamp += timedelta(hours=6)
                    value2 = hk_v_winds[timestamp.strftime("%Y%m%d_%H_%M")]
                    weight = 1 - (row["ISO_TIME_HOUR"] - (row["ISO_TIME_HOUR"] // 6)*6)/6.0
                    value = value1 * weight + value2 * (1-weight)
                    data_row.append(value)
                except KeyError as e:
                    cannot_build_past_track = True
                    num_series_ng += 1
                    break
            # others
            if key in u200:
                data_row.append(u200[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
            if key in u500:
                data_row.append(u500[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
            if key in v500:
                data_row.append(v500[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
            # easm
            if time_key in easm:
                data_row.append(easm[time_key])
            else:
                try:
                    timestamp = datetime(row["SEASON"], row["ISO_TIME_MONTH"], row["ISO_TIME_DAY"], row["ISO_TIME_HOUR"], 0, 0)
                    number_of_hours_to_go_back = int(row["ISO_TIME_HOUR"] % 6)
                    timestamp -= timedelta(hours=number_of_hours_to_go_back)
                    value1 = easm[timestamp.strftime("%Y%m%d_%H_%M")]
                    timestamp += timedelta(hours=6)
                    value2 = easm[timestamp.strftime("%Y%m%d_%H_%M")]
                    weight = 1 - (row["ISO_TIME_HOUR"] - (row["ISO_TIME_HOUR"] // 6)*6)/6.0
                    value = value1 * weight + value2 * (1-weight)
                    data_row.append(value)
                except KeyError as e:
                    cannot_build_past_track = True
                    num_series_ng += 1
                    break
            
            # vorticity
            if key in vort850:
                data_row.append(vort850[key] * 1e6)
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
                
            # geopotential height-related
            if time_key in westerly:
                data_row.append(westerly[time_key])
            else:
                try:
                    timestamp = datetime(row["SEASON"], row["ISO_TIME_MONTH"], row["ISO_TIME_DAY"], row["ISO_TIME_HOUR"], 0, 0)
                    number_of_hours_to_go_back = int(row["ISO_TIME_HOUR"] % 6)
                    timestamp -= timedelta(hours=number_of_hours_to_go_back)
                    value1 = westerly[timestamp.strftime("%Y%m%d_%H_%M")]
                    timestamp += timedelta(hours=6)
                    value2 = westerly[timestamp.strftime("%Y%m%d_%H_%M")]
                    weight = 1 - (row["ISO_TIME_HOUR"] - (row["ISO_TIME_HOUR"] // 6)*6)/6.0
                    value = value1 * weight + value2 * (1-weight)
                    data_row.append(value)
                except KeyError as e:
                    cannot_build_past_track = True
                    num_series_ng += 1
                    break
            if time_key in wnpsh_area:
                data_row.append(int(wnpsh_area[time_key]))
            else:
                try:
                    timestamp = datetime(row["SEASON"], row["ISO_TIME_MONTH"], row["ISO_TIME_DAY"], row["ISO_TIME_HOUR"], 0, 0)
                    number_of_hours_to_go_back = int(row["ISO_TIME_HOUR"] % 6)
                    timestamp -= timedelta(hours=number_of_hours_to_go_back)
                    value1 = wnpsh_area[timestamp.strftime("%Y%m%d_%H_%M")]
                    timestamp += timedelta(hours=6)
                    value2 = wnpsh_area[timestamp.strftime("%Y%m%d_%H_%M")]
                    weight = 1 - (row["ISO_TIME_HOUR"] - (row["ISO_TIME_HOUR"] // 6)*6)/6.0
                    value = value1 * weight + value2 * (1-weight)
                    data_row.append(round(value)) # round it to get better average
                except KeyError as e:
                    cannot_build_past_track = True
                    num_series_ng += 1
                    break
            if time_key in wnpsh_intensity:
                data_row.append(wnpsh_intensity[time_key])
            else:
                try:
                    timestamp = datetime(row["SEASON"], row["ISO_TIME_MONTH"], row["ISO_TIME_DAY"], row["ISO_TIME_HOUR"], 0, 0)
                    number_of_hours_to_go_back = int(row["ISO_TIME_HOUR"] % 6)
                    timestamp -= timedelta(hours=number_of_hours_to_go_back)
                    value1 = wnpsh_intensity[timestamp.strftime("%Y%m%d_%H_%M")]
                    timestamp += timedelta(hours=6)
                    value2 = wnpsh_intensity[timestamp.strftime("%Y%m%d_%H_%M")]
                    weight = 1 - (row["ISO_TIME_HOUR"] - (row["ISO_TIME_HOUR"] // 6)*6)/6.0
                    value = value1 * weight + value2 * (1-weight)
                    data_row.append(value)
                except KeyError as e:
                    cannot_build_past_track = True
                    num_series_ng += 1
                    break
            if time_key in wnpsh_extension:
                data_row.append(int(wnpsh_extension[time_key]))
            else:
                try:
                    timestamp = datetime(row["SEASON"], row["ISO_TIME_MONTH"], row["ISO_TIME_DAY"], row["ISO_TIME_HOUR"], 0, 0)
                    number_of_hours_to_go_back = int(row["ISO_TIME_HOUR"] % 6)
                    timestamp -= timedelta(hours=number_of_hours_to_go_back)
                    value1 = wnpsh_extension[timestamp.strftime("%Y%m%d_%H_%M")]
                    timestamp += timedelta(hours=6)
                    value2 = wnpsh_extension[timestamp.strftime("%Y%m%d_%H_%M")]
                    weight = 1 - (row["ISO_TIME_HOUR"] - (row["ISO_TIME_HOUR"] // 6)*6)/6.0
                    value = value1 * weight + value2 * (1-weight)
                    data_row.append(round(value))
                except KeyError as e:
                    cannot_build_past_track = True
                    num_series_ng += 1
                    break
                
            # potential temp
            if key in pott:
                data_row.append(pott[key])
            else:
                cannot_build_past_track = True
                num_series_ng += 1
                break
                
        if not cannot_build_past_track:
            dataArray.append(data_row) 
        # else:
           # print("A record of TC {0} ({1}) contains missing data and is unusable :(".format(row['NAME'], row['SEASON']))
            
    num_storms_processed += 1
    if num_storms_processed % 100 == 0:
        print("Processed {0} storms out of {1}, {2} records unusable.".format(num_storms_processed, storms.shape[0], num_series_ng))
    
print()
ds = pd.DataFrame(dataArray, columns=columns)
print('Number of TC best track - warning record sequences: {0}'.format(ds.shape[0]))

Number of storms identified in the dataset: 591
Creating time series from dynamical data, best track data sequences and warning records.
Processed 100 storms out of 591, 2170 records unusable.
Processed 200 storms out of 591, 4726 records unusable.
Processed 300 storms out of 591, 5723 records unusable.
Processed 400 storms out of 591, 5723 records unusable.
Processed 500 storms out of 591, 5725 records unusable.

Number of TC best track - warning record sequences: 6955


### Phew. Let's check if it screwed up one way or another

In [26]:
# inspections #

# In principle, we should not get more than 8900 rows in the dataset 
# (as capped by the number of vorticity/humidity data records).
# Amazingly, some 9308 sequences are possible, which is decent but suspicious.
print(num_series_ng)
print(best_track.shape)
print(best_track.shape[0] - num_series_ng)

# note that the baseline has beyond 45000 rows and ~20 columns. This time we have 153 columns and a much smaller number of rows.
print(ds.shape)

5725
(15033, 25)
9308
(6955, 153)


In [27]:
# null checking
ds[ds.isnull().any(axis=1)]

,LOW_IMPACT,MID_IMPACT,BIG_IMPACT,DIRECT_STRIKE,MM00,DD00,MI_STATUS00,LI_STATUS00,SI_STATUS00,DS_STATUS00,...,U20024,U50024,V50024,EASM24,VORT24,WESTERLY24,SH_AREA24,SH_INT24,SH_EXT24,POTT24


In [28]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6955 entries, 0 to 6954
Columns: 153 entries, LOW_IMPACT to POTT24
dtypes: bool(24), float64(90), int32(29), int64(10)
memory usage: 6.2 MB


In [34]:
pd.set_option('display.max_columns', None)
ds.head()

,LOW_IMPACT,MID_IMPACT,BIG_IMPACT,DIRECT_STRIKE,MM00,DD00,MI_STATUS00,LI_STATUS00,SI_STATUS00,DS_STATUS00,DIST00,AZM00,SPEED00,DIR00,VMAX00,DVMAX00,ULVWS00,MLVWS00,HI_HUMID00,LO_HUMID00,STEMP00,UTEMP00,U_HK00,V_HK00,U20000,U50000,V50000,EASM00,VORT00,WESTERLY00,SH_AREA00,SH_INT00,SH_EXT00,POTT00,MM06,DD06,MI_STATUS06,LI_STATUS06,SI_STATUS06,DS_STATUS06,DIST06,AZM06,SPEED06,DIR06,VMAX06,DVMAX06,ULVWS06,MLVWS06,HI_HUMID06,LO_HUMID06,STEMP06,UTEMP06,U_HK06,V_HK06,U20006,U50006,V50006,EASM06,VORT06,WESTERLY06,SH_AREA06,SH_INT06,SH_EXT06,POTT06,MM12,DD12,MI_STATUS12,LI_STATUS12,SI_STATUS12,DS_STATUS12,DIST12,AZM12,SPEED12,DIR12,VMAX12,DVMAX12,ULVWS12,MLVWS12,HI_HUMID12,LO_HUMID12,STEMP12,UTEMP12,U_HK12,V_HK12,U20012,U50012,V50012,EASM12,VORT12,WESTERLY12,SH_AREA12,SH_INT12,SH_EXT12,POTT12,MM18,DD18,MI_STATUS18,LI_STATUS18,SI_STATUS18,DS_STATUS18,DIST18,AZM18,SPEED18,DIR18,VMAX18,DVMAX18,ULVWS18,MLVWS18,HI_HUMID18,LO_HUMID18,STEMP18,UTEMP18,U_HK18,V_HK18,U20018,U50018,V50018,EASM18,VORT18,WESTERLY18,SH_AREA18,SH_INT18,SH_EXT18,POTT18,MM24,DD24,MI_STATUS24,LI_STATUS24,SI_STATUS24,DS_STATUS24,DIST24,AZM24,SPEED24,DIR24,VMAX24,ULVWS24,MLVWS24,HI_HUMID24,LO_HUMID24,STEMP24,UTEMP24,U_HK24,V_HK24,U20024,U50024,V50024,EASM24,VORT24,WESTERLY24,SH_AREA24,SH_INT24,SH_EXT24,POTT24
0,False,False,False,False,1,14,False,False,False,False,1108.140101,182.684092,12,247,40,5,0.0,0.0,49.180557,49.180557,272.87500,225.08333,18.4452,26.061701,53.363194,53.363194,12.060417,36.532387,90.75,416.56293,0,0.0000,0,271.95000,1,14,False,False,False,False,1054.488536,175.697325,12,248,35,5,0.0,0.0,49.180557,49.180557,272.87500,225.08333,18.4452,26.061701,53.363194,53.363194,12.060417,36.532387,90.75,416.56293,0,0.0000,0,271.95000,1,13,False,False,False,False,1025.809285,168.831691,11,252,30,0,0.0,0.0,49.180557,49.180557,272.87500,225.08333,18.4452,26.061701,53.363194,53.363194,12.060417,36.532387,90.75,416.56293,0,0.0000,0,271.95000,1,13,False,False,False,False,1019.088182,162.463138,10,276,30,5,0.0,0.0,49.180557,49.180557,272.87500,225.08333,18.4452,26.061701,53.363194,53.363194,12.060417,36.532387,90.75,416.56293,0,0.0000,0,271.95000,1,13,False,False,False,False,1110.394048,157.346520,11,314,25,0.0,0.0,49.180557,49.180557,272.87500,225.08333,18.4452,26.061701,53.363194,53.363194,12.060417,36.532387,90.75,416.56293,0,0.0000,0,271.95000
1,False,False,False,False,1,14,False,False,False,False,1172.821661,187.920127,12,224,35,-5,0.0,0.0,59.215280,59.215280,263.75000,224.53334,22.7799,25.313200,51.670834,51.670834,15.001389,35.627434,87.25,405.20620,0,0.0000,0,271.00000,1,14,False,False,False,False,1108.140101,182.684092,12,247,40,5,0.0,0.0,59.215280,59.215280,263.75000,224.53334,22.7799,25.313200,51.670834,51.670834,15.001389,35.627434,87.25,405.20620,0,0.0000,0,271.00000,1,14,False,False,False,False,1054.488536,175.697325,12,248,35,5,0.0,0.0,59.215280,59.215280,263.75000,224.53334,22.7799,25.313200,51.670834,51.670834,15.001389,35.627434,87.25,405.20620,0,0.0000,0,271.00000,1,13,False,False,False,False,1025.809285,168.831691,11,252,30,0,0.0,0.0,59.215280,59.215280,263.75000,224.53334,22.7799,25.313200,51.670834,51.670834,15.001389,35.627434,87.25,405.20620,0,0.0000,0,271.00000,1,13,False,False,False,False,1019.088182,162.463138,10,276,30,0.0,0.0,59.215280,59.215280,263.75000,224.53334,22.7799,25.313200,51.670834,51.670834,15.001389,35.627434,87.25,405.20620,0,0.0000,0,271.00000
2,False,False,False,False,1,14,False,False,False,False,1337.412594,190.330576,13,203,30,-5,0.0,0.0,67.784720,67.784720,260.07498,223.30972,21.4434,28.030500,46.818054,46.818054,11.184722,35.374070,101.00,397.72840,0,0.0000,0,273.72500,1,14,False,False,False,False,1172.821661,187.920127,12,224,35,-5,0.0,0.0,67.784720,67.784720,260.07498,223.30972,21.4434,28.030500,46.818054,46.818054,11.184722,35.374070,101.00,397.72840,0,0.0000,0,273.72500,1,14,False,False,False,False,1108.140101,182.684092,12,247,40,5,0.0,0.0,67.784720,67.784720,260.07498,223.30972,21.4434,28.030500,46.818054,46.818054,11

### Observations
- high and low level humidities are oddly close

- u200 and u500 are very close

In [35]:
# save to file
output_pandas_to_file(ds, "./experimental_datasets/experimental_dataset_tsnv_{0}.gz".format(PAST_TRACK_LIMIT))

Reading file ./experimental_datasets/experimental_dataset_tsnv_24.gz ......Done!
